In [ ]:
%cd /content/ai-competition-baselines
!python -m src.stage1_binary.infer \
  --config configs/stage1_binary.yaml \
  --data_root "{DATA_ROOT}" \
  --ckpt runs/stage1_binary/best_stage1.pt \
  --test_dir images/test \
  --out submissions/stage1.csv

!head -n 5 submissions/stage1.csv

In [ ]:
# CELL 20: Run Inference on Test Set
DATA_ROOT = "/content/drive/MyDrive/datasets/defects_stage2"
TEST_DIR = "images/test"
OUT_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_test.csv"
CKPT_PATH = "/content/drive/MyDrive/project_stage2/runs/stage2_multilabel/best_stage2.pt"
CONFIG_PATH = "/content/drive/MyDrive/project_stage2/configs/stage2_multilabel.yaml"

!python -m src.stage2_multilabel.infer \
    --config "{CONFIG_PATH}" \
    --data_root "{DATA_ROOT}" \
    --ckpt "{CKPT_PATH}" \
    --test_dir "{TEST_DIR}" \
    --out "{OUT_CSV}"

print("\nTest predictions (first 10 rows):")
import pandas as pd
df = pd.read_csv(OUT_CSV)
df.head(10)

In [ ]:
# CELL 21: Run Inference on Validation Set
DATA_ROOT = "/content/drive/MyDrive/datasets/defects_stage2"
VAL_DIR = "images/val"
OUT_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv"
CKPT_PATH = "/content/drive/MyDrive/project_stage2/runs/stage2_multilabel/best_stage2.pt"
CONFIG_PATH = "/content/drive/MyDrive/project_stage2/configs/stage2_multilabel.yaml"

!python -m src.stage2_multilabel.infer \
    --config "{CONFIG_PATH}" \
    --data_root "{DATA_ROOT}" \
    --ckpt "{CKPT_PATH}" \
    --test_dir "{VAL_DIR}" \
    --out "{OUT_CSV}"

print("\nValidation predictions (first 10 rows):")
df = pd.read_csv(OUT_CSV)
df.head(10)

In [ ]:
# CELL 22: Evaluate on Validation Set (Base Metrics)
DATA_ROOT = "/content/drive/MyDrive/datasets/defects_stage2"
GT_CSV = f"{DATA_ROOT}/val.csv"
PRED_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv"

!python -m src.stage2_multilabel.eval \
    --gt "{GT_CSV}" \
    --pred "{PRED_CSV}"

In [ ]:
# CELL 23: Evaluate with Threshold Tuning
DATA_ROOT = "/content/drive/MyDrive/datasets/defects_stage2"
GT_CSV = f"{DATA_ROOT}/val.csv"
PRED_CSV = "/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv"

!python -m src.stage2_multilabel.eval \
    --gt "{GT_CSV}" \
    --pred "{PRED_CSV}" \
    --tune_thresholds

In [ ]:
# CELL 24: View Training Summary
import json

with open('/content/drive/MyDrive/project_stage2/runs/stage2_multilabel/train_summary.json', 'r') as f:
    summary = json.load(f)

print("Training History:")
print("-" * 100)
for epoch_data in summary['history']:
    print(f"Epoch {epoch_data['epoch']}: "
          f"train_loss={epoch_data['train_loss']:.4f}, "
          f"val_loss={epoch_data['val_loss']:.4f}, "
          f"mAP={epoch_data['map']:.4f}, "
          f"macro_f1={epoch_data['macro_f1']:.4f}")

print("-" * 100)
print(f"Best mAP: {summary['best_primary']:.4f}")

In [ ]:
# CELL 25: Analyze Label Distribution
import pandas as pd
import matplotlib.pyplot as plt

val_df = pd.read_csv('/content/drive/MyDrive/datasets/defects_stage2/val.csv')
pred_df = pd.read_csv('/content/drive/MyDrive/project_stage2/submissions/stage2_val.csv')
label_cols = ["Tucked", "Wrinkles", "Items"]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, label in enumerate(label_cols):
    gt_counts = val_df[label].value_counts().sort_index()
    axes[idx].bar(['Absent', 'Present'], gt_counts.values, color=['#3498db', '#e74c3c'])
    axes[idx].set_title(f'{label} Distribution', fontweight='bold', fontsize=12)
    axes[idx].set_ylabel('Count')
    axes[idx].grid(axis='y', alpha=0.3)
    for i, v in enumerate(gt_counts.values):
        axes[idx].text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/project_stage2/label_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# CELL 26: Visualize Sample Predictions
from PIL import Image
import matplotlib.pyplot as plt

merged = val_df.merge(pred_df, on='image_id')
sample_preds = merged.sample(n=min(6, len(merged)))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()
label_cols = ["Tucked", "Wrinkles", "Items"]
label_names = {"Tucked": "Tucked", "Wrinkles": "Wrinkles", "Items": "Items"}
DATA_ROOT = "/content/drive/MyDrive/datasets/defects_stage2"

for idx, (_, row) in enumerate(sample_preds.iterrows()):
    img_path = f"{DATA_ROOT}/images/val/{row['image_id']}"
    img = Image.open(img_path)
    axes[idx].imshow(img)

    title_lines = []
    for label in label_cols:
        gt_val = int(row[label])
        prob_val = row[f'prob_{label}']
        pred_val = 1 if prob_val >= 0.5 else 0
        color = '✓' if gt_val == pred_val else '✗'
        title_lines.append(f"{label_names[label]}: GT={gt_val} Pred={pred_val} ({prob_val:.2f}) {color}")
    title = '\n'.join(title_lines)
    axes[idx].set_title(title, fontsize=9, ha='left', loc='left')
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/project_stage2/sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()